## Amazon Bedrock Guardrails: 
Amazon Bedrock Guardrails provides robust content filtering for generative AI applications. This customizable framework helps enterprises implement safety protocols for their AI applications by setting up filters to prevent harmful, inaccurate, or sensitive content. Guardrails enables control over topics, content appropriateness, and PII handling through configurable policies. Organizations can define what information should be blocked, filtered, or anonymized based on specific requirements. Built with regulatory compliance in mind, Guardrails allows developers to establish boundaries that keep AI interactions safe while maintaining functionality and utility. It integrates seamlessly with Bedrock's foundation models, providing protection without sacrificing performance or user experience.

in this notebook: we will perform the following 

## Basic Setup
- **UUID Generation**: Creates a unique client request token to ensure idempotency (prevents accidental duplicate guardrail creation)
- **Name and Description**: Sets clear identifiers for the guardrail's purpose ("AdvancedRagWorkshopGuardrails" focused on SEC filing content)
- **Blocked Messages**: Customized messages displayed when input or output is blocked by guardrails

## Topic Policy Configuration
- **Financial Advice (DENY)**: Blocks any content that recommends investments or financial decisions
- **Legal Interpretation (DENY)**: Prevents the model from interpreting legal or regulatory requirements
- **Future Performance**: There's a commented-out policy that would block speculation beyond disclosed forward-looking statements

## Content Policy Configuration
Sets high-strength filters for both input and output on:
- **HATE**: Blocks hateful content
- **INSULTS**: Filters insulting language
- **SEXUAL**: Prevents sexual content
- **VIOLENCE**: Blocks violent content
- **MISCONDUCT**: Filters content promoting illegal or harmful activities
- **PROMPT_ATTACK**: Only filters input (HIGH) but not output (NONE) for prompt injection attempts

## Contextual Grounding Policy
Sets thresholds for ensuring responses remain grounded to source material:
- **GROUNDING**: 0.1 threshold requires responses to be based on the provided context/documents
- **RELEVANCE**: 0.1 threshold ensures responses are relevant to the user's query

## Word Policy Configuration
Blocks specific words deemed inappropriate for the application:

## Sensitive Information Policy
### PII Entities
Configures ANONYMIZE action (redacts/masks content) for a comprehensive list of personal identifiable information types:
- Personal identifiers (NAME, EMAIL, PHONE, ADDRESS, AGE)
- Financial data (CREDIT_DEBIT_CARD details, BANK ACCOUNT numbers)
- Government IDs (SOCIAL_SECURITY_NUMBER, DRIVER_ID, PASSPORT_NUMBER)
- Technical identifiers (IP_ADDRESS, MAC_ADDRESS)
- Authentication data (PASSWORD, PIN, USERNAME)
- And many other PII types

### Regex Patterns
Custom regex patterns to catch domain-specific sensitive information:
- **Medical Record Numbers**: Pattern matching "123-456-7890" format
- **Insurance Numbers**: Pattern matching two letters followed by 7 digits

## Tags
Metadata for organizing and tracking the guardrail:
- **Environment**: "Production"
- **Department**: "Finance"

## Response Handling
Captures and outputs the:
- **Guardrail ID**: Unique identifier for the created guardrail
- **Guardrail ARN**: Full Amazon Resource Name
- **Version**: Version number of the created guardrail

## 1. Import Required Libraries

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError, BotoCoreError

# Initialize the Amazon Bedrock client in the us-west-2 region
bedrock = boto3.client('bedrock', region_name='us-west-2')


## 2. Load Configuration Variables

In [ ]:
with open("../Lab 1/variables.json", "r") as f:
    variables = json.load(f)

variables

## 3. Create a Guardrail

In [ ]:
import uuid
import boto3

# Initialize the Amazon Bedrock client in the us-west-2 region
bedrock = boto3.client('bedrock', region_name='us-west-2')

# Generate a unique client request token
client_request_token = str(uuid.uuid4())

# Create a Guardrail with specific filtering and compliance policies
response = bedrock.create_guardrail(
    name="AdvancedRagWorkshopGuardrails",
    description="Restrict responses to SEC filing content only",
    blockedInputMessaging="This request cannot be processed due to safety protocols.",
    blockedOutputsMessaging="Response modified per compliance guidelines.",

    # Topic-based restrictions (e.g., denying financial advice)
    topicPolicyConfig={
        'topicsConfig': [
            {'name': 'financial-advice', 'definition': 'Any recommendations about investments or financial decisions', 'type': 'DENY'},
            {'name': 'legal-interpretation', 'definition': 'Interpretation of legal or regulatory requirements', 'type': 'DENY'}
        ]
    },

    # Content filtering policies (e.g., blocking hate speech, violence)
    contentPolicyConfig={
        'filtersConfig': [
            {'type': 'HATE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'INSULTS', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'SEXUAL', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'VIOLENCE', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'MISCONDUCT', 'inputStrength': 'HIGH', 'outputStrength': 'HIGH'},
            {'type': 'PROMPT_ATTACK', 'inputStrength': 'HIGH', 'outputStrength': 'NONE'}
        ]
    },

    # Contextual grounding policies (e.g., ensuring responses stay relevant)
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {'type': 'GROUNDING', 'threshold': 0.1},
            {'type': 'RELEVANCE', 'threshold': 0.1}
        ]
    },

    # List of restricted words related to financial risks
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'material weakness'}, {'text': 'undisclosed liabilities'}, {'text': 'shareholder lawsuit'},
            {'text': 'SEC investigation'}, {'text': 'accounting irregularities'}, {'text': 'restate earnings'},
            {'text': 'liquidity crisis'}, {'text': 'bankruptcy risk'}, {'text': 'fraudulent activity'},
            {'text': 'insider trading'}
        ]
    },

    # Sensitive data anonymization (e.g., PII, financial information)
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'NAME', 'action': 'ANONYMIZE'}, {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'}, {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'ANONYMIZE'},
            {'type': 'ADDRESS', 'action': 'ANONYMIZE'}, {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'ANONYMIZE'},
            {'type': 'PASSWORD', 'action': 'ANONYMIZE'}, {'type': 'IP_ADDRESS', 'action': 'ANONYMIZE'}
        ],
        'regexesConfig': [
            {'name': 'stock_ticker_with_price', 'description': 'Stock ticker with price pattern',
             'pattern': '\\b[A-Z]{1,5}\\s*[@:]\\s*\\$?\\d+(\\.\\d{1,2})?\\b', 'action': 'ANONYMIZE'},
            {'name': 'financial_figures', 'description': 'Large financial figures in billions/millions',
             'pattern': '\\$\\s*\\d+(\\.\\d+)?\\s*(billion|million|B|M)\\b', 'action': 'ANONYMIZE'}
        ]
    },

    # Tags for environment tracking
    tags=[
        {'key': 'Environment', 'value': 'Production'},
        {'key': 'Department', 'value': 'Finance'}
    ],
    clientRequestToken=client_request_token
)

# Retrieve and print the Guardrail ID, ARN, and version
guardrail_id = response['guardrailId']
print(f"Guardrail ID: {guardrail_id}")
print(f"Guardrail ARN: {response['guardrailArn']}")
print(f"Version: {response['version']}")


## 4. Save Guardrail ID to Configuration File

In [ ]:
# Update the variables dictionary with the new Guardrail ID
variables["guardrail_id"] = guardrail_id

# Save updated variables to a JSON file
with open("../Lab 1/variables.json", "w") as f:
    json.dump(variables, f, indent=4, default=str) 

## 5. Create a Published Version of the Guardrail

In [ ]:
# First create a published version
version_response = bedrock.create_guardrail_version(
    guardrailIdentifier=response['guardrailId'],
    description="Production version 1.0"
)



## 6. Retrieve and Save Guardrail Version

In [ ]:
guardrail_version=version_response['version']
guardrail_version


## 7. Save Guardrail Version to Configuration File

In [ ]:
# Update the variables dictionary with the new Guardrail Version
variables["guardrail_version"] = guardrail_version

# Save updated variables to a JSON file
with open("../Lab 1/variables.json", "w") as f:
    json.dump(variables, f, indent=4, default=str) 

> **Note**: After creating the guardrail, we recommend exploring its settings in the Amazon Bedrock console. The console provides a visual interface to understand how the guardrail is configured and allows you to test it interactively.
> 
> **[➡️ View your Guardrails in the AWS Console](https://us-west-2.console.aws.amazon.com/bedrock/home?region=us-west-2#/guardrails)**
>
> In the console, you can:
> - See all configured policies (topic filters, content filters, sensitive information policies)
> - Test sample prompts against your guardrail
> - View guardrail versions and their differences
> - Monitor usage metrics and blocked content statistics
> - Make adjustments to policies as needed